# Derivative Estimation using Local Polynomial Fitting
This notebook implements derivative estimation using local polynomial fitting based on De Brabanter's method.

## Import Required Libraries
We'll import the necessary Python libraries for numerical computations and visualization.

In [ ]:
import numpy as np
import scipy.linalg as la
from scipy.stats import norm
import matplotlib.pyplot as plt
plt.style.use('seaborn')

## Generate Synthetic Data
Generate test data from a known function with added noise to evaluate the derivative estimation method.

In [ ]:
# Generate true function and its derivative
def true_function(x):
    return np.sin(2 * np.pi * x) * np.exp(-x/2)

def true_derivative(x):
    return np.exp(-x/2) * (2 * np.pi * np.cos(2 * np.pi * x) - 0.5 * np.sin(2 * np.pi * x))

# Generate noisy observations
n_points = 100
x = np.linspace(0, 4, n_points)
y = true_function(x) + np.random.normal(0, 0.1, n_points)

## Define Local Polynomial Fitting Function
Implement the local polynomial fitting algorithm using weighted least squares estimation.

In [ ]:
def local_polynomial_fit(x, y, x0, h, p=1):
    """
    Local polynomial fitting at point x0
    
    Parameters:
    x: input points
    y: observed values
    x0: point at which to estimate
    h: bandwidth
    p: polynomial degree
    """
    # Calculate weights
    weights = norm.pdf((x - x0) / h)
    
    # Create design matrix
    X = np.vstack([((x - x0) ** i) for i in range(p + 1)]).T
    W = np.diag(weights)
    
    # Weighted least squares
    beta = la.solve(X.T @ W @ X, X.T @ W @ y)
    
    # Return estimate and derivative
    return beta[0], beta[1] if p >= 1 else None

## Estimate Derivatives
Apply the local polynomial fitting method to estimate derivatives across the input range.

In [ ]:
# Estimate derivatives
x_eval = np.linspace(0, 4, 200)
y_est = []
dy_est = []

h = 0.2  # bandwidth parameter
for x0 in x_eval:
    f_est, df_est = local_polynomial_fit(x, y, x0, h, p=2)
    y_est.append(f_est)
    dy_est.append(df_est)

y_est = np.array(y_est)
dy_est = np.array(dy_est)

## Visualize Results
Compare the estimated derivatives with the true derivatives and plot the results.

In [ ]:
# Create subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Plot function estimates
ax1.scatter(x, y, alpha=0.5, label='Noisy data')
ax1.plot(x_eval, true_function(x_eval), 'r-', label='True function')
ax1.plot(x_eval, y_est, 'b--', label='Estimated function')
ax1.legend()
ax1.set_title('Function Estimation')

# Plot derivative estimates
ax2.plot(x_eval, true_derivative(x_eval), 'r-', label='True derivative')
ax2.plot(x_eval, dy_est, 'b--', label='Estimated derivative')
ax2.legend()
ax2.set_title('Derivative Estimation')

plt.tight_layout()
plt.show()